[View in Colaboratory](https://colab.research.google.com/github/ayan59dutta/Handwritten-Text-Extractor-using-NN/blob/master/Handwritten_Character_Recognizer_ANN.ipynb)

In [0]:
import time
start = time.time()
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import zipfile
print('Packages imported in', time.time()-start, 'seconds...')

Packages imported in 0.8629271984100342 seconds...


In [0]:
# Do not run this cell if dataset is already present in folder

!pip install gdown
!gdown https://drive.google.com/uc?id=1JLBWOyxNWDxUnIarBVMdAv460z1ICkuh

  Running setup.py bdist_wheel for gdown ... - done
  Stored in directory: /root/.cache/pip/wheels/97/90/fa/25654eb65da3e6da7752db71a164e0eb8f7a6fb4335eeb46ab
Successfully built gdown
Downloading...
From: https://drive.google.com/uc?id=1JLBWOyxNWDxUnIarBVMdAv460z1ICkuh
To: /content/nist_by_class.zip
439MB [00:02, 213MB/s]


In [0]:
# def download_file_from_google_drive(id='1JLBWOyxNWDxUnIarBVMdAv460z1ICkuh', destination='nist_by_class.zip'):
#     def get_confirm_token(response):
#         for key, value in response.cookies.items():
#             if key.startswith('download_warning'):
#                 return value

#         return None

#     def save_response_content(response, destination):
#         CHUNK_SIZE = 32768

#         with open(destination, "wb") as f:
#             for chunk in response.iter_content(CHUNK_SIZE):
#                 if chunk: # filter out keep-alive new chunks
#                     f.write(chunk)

#     URL = "https://docs.google.com/uc?export=download"

#     session = requests.Session()

#     response = session.get(URL, params = { 'id' : id }, stream = True)
#     token = get_confirm_token(response)

#     if token:
#         params = { 'id' : id, 'confirm' : token }
#         response = session.get(URL, params = params, stream = True)

#     save_response_content(response, destination)

# download_file_from_google_drive()

In [0]:
# Insert absolute path of the directory in which dataset is present here, if applicable

abs_path = ''

start = time.time()
with zipfile.ZipFile(abs_path+'nist_by_class.zip', 'r') as zip_ref:
    zip_ref.extractall(abs_path+'nist_by_class')
print('Dataset unzipped in', time.time()-start, 'seconds...')
# !rm 'nist_by_class.zip'

Dataset unzipped in 106.11546540260315 seconds...


In [0]:
class_info = {}
for i in range(48, 58):
    class_info[i-48] = (hex(i)[2:], i, chr(i))
for i in range(65, 65+26):
    class_info[i-65+10] = (hex(i)[2:], i, chr(i))
for i in range(97, 97+26):
    class_info[i-97+36] = (hex(i)[2:], i, chr(i))

In [0]:
start = time.time()
data_list = []
# data_list = np.array([]).reshape(0, 4097)
total_count = 0

for i in class_info:
    print(class_info[i], '\t', time.time()-start, 'seconds', end='\t')
    path = abs_path + 'nist_by_class/' + class_info[i][0] + '/train_' + class_info[i][0]
    count = 0
    for img in os.listdir(path):
        im = Image.open(path + '/' + img).convert('L').crop((32, 32, 96, 96))
        im_arr = np.array(im, dtype='float32') / 255.0
        flat_im_arr = im_arr.flatten()
        flat_im_arr_labelled = np.insert(flat_im_arr, 0, i)
        data_list.append(flat_im_arr_labelled)
#         temp_data_list = data_list.copy()
#         del data_list
#         data_list = np.append(temp_data_list, flat_im_arr_labelled.reshape(1, 4097), axis=0)
        count += 1
    print(count, 'samples')
#     !rm -rf 'by_class/' + class_info[i][0]
#     !rm -rf class_info[i][0]
    total_count += count

# !rm -rf 'by_class'
data = np.array(data_list, dtype='float32')

print('Total number of samples =', total_count)
print('Shape of final data =', data.shape)

print('All sample images processed in', time.time()-start, 'seconds...')

('30', 48, '0') 	 0.00018143653869628906 seconds	34803 samples
('31', 49, '1') 	 16.88612699508667 seconds	38049 samples
('32', 50, '2') 	 34.999566078186035 seconds	34184 samples
('33', 51, '3') 	 51.77231311798096 seconds	35293 samples
('34', 52, '4') 	 69.25126457214355 seconds	33432 samples
('35', 53, '5') 	 85.60997915267944 seconds	31067 samples
('36', 54, '6') 	 101.0075945854187 seconds	34079 samples
('37', 55, '7') 	 118.14982151985168 seconds	35796 samples
('38', 56, '8') 	 135.94138932228088 seconds	33884 samples
('39', 57, '9') 	 157.61754631996155 seconds	33720 samples
('41', 65, 'A') 	 194.5521948337555 seconds	7010 samples
('42', 66, 'B') 	 201.87365674972534 seconds	4091 samples
('43', 67, 'C') 	 206.26549696922302 seconds	11315 samples
('44', 68, 'D') 	 218.03602838516235 seconds	4945 samples
('45', 69, 'E') 	 223.05316925048828 seconds	5420 samples
('46', 70, 'F') 	 228.76392030715942 seconds	10203 samples
('47', 71, 'G') 	 239.36686301231384 seconds	2575 samples
('48

In [0]:
start = time.time()
data_shuffled = np.random.shuffle(data)
print('Data shuffled in', time.time()-start, 'seconds...')

Data shuffled in 3140.593427658081 seconds...


In [0]:
X_train = data[:627000, 1:]
Y_train = data[:627000, 0]
X_test = data[627000:, 1:]
Y_test = data[627000:, 0]
del data

In [0]:
model = tf.keras.models.Sequential()
layer = tf.keras.layers.Dense(256, input_shape=[64*64*1])
model.add(layer)
model.add(tf.keras.layers.Dense(192))
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Dense(62, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

In [0]:
Y_train = tf.keras.utils.to_categorical(Y_train)
# X_train = X_train.astype('float32') / 255.0

In [0]:
model.fit(X_train, Y_train, epochs=5)

Epoch 1/1
627000/627000 [==============================] - 2217s 4ms/step - loss: 15.3387


In [0]:
start = time.time()
loss, acc = new_model.evaluate(X_test, Y_test)
print('Accuracy on test set :', acc*100, '%')
print('Loss on test set :', loss')
print('Model evaluated on test set in', time.time()-start, 'seconds...')

In [0]:
start = time.time()
model.save('./saved_model/HCRANN.h5')
print('Model saved to disk in', time.time()-start, 'seconds...')

In [0]:
to_predict = X_test[0]

pred = model.predict(X_test[0].reshape(4096,))
print(class_info[np.argmax(pred)][-1])

plt.axis('off')
plt.imshow(to_predict.reshape(64, 64))
plt.show()

ValueError: Error when checking input: expected dense_input to have shape (4096,) but got array with shape (1,)

In [0]:
X_test[0].shape

(4096,)

In [0]:
to_predict.shape

(4096,)

In [0]:
pred

NameError: name 'pred' is not defined